# Overview

## Setup

In [ ]:
from langchain_google_vertexai import ChatVertexAI
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_chroma import Chroma

llm = ChatVertexAI(model="gemini-1.0-pro")
embeddings = VertexAIEmbeddings(model="text-embedding-004")
vector_store = Chroma(embedding_function=embeddings)

In [ ]:
import bs4
import uuid
from typing import Annotated, Sequence
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, BaseMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import START, END, StateGraph, MessagesState
from typing_extensions import List, TypedDict
from IPython.display import Image, display

## QA RAG model
see: https://python.langchain.com/docs/tutorials/rag/

### Pipeline for ingesting data from a source and indexing it (by semantic search for our case)

1. Load data with a data loader
2. Break large documents into smaller chunks with text splitters to fit into model's finite context window
3. Use vector store and embeddings model to store and index the splits

In [ ]:
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/", # general llm agent blog for the curious
               # help llm make sense of functions
                "https://xrpl-py.readthedocs.io/en/stable/source/snippets.html",
                "https://xrpl-py.readthedocs.io/en/stable/source/xrpl.models.html",
                "https://xrpl-py.readthedocs.io/en/stable/source/xrpl.wallet.html",
                "https://xrpl-py.readthedocs.io/en/stable/source/xrpl.clients.html",
                "https://xrpl-py.readthedocs.io/en/stable/source/xrpl.transaction.html",
                "https://xrpl-py.readthedocs.io/en/stable/source/xrpl.ledger.html",
                "https://xrpl-py.readthedocs.io/en/stable/source/xrpl.core.addresscodec.html",
                "https://xrpl-py.readthedocs.io/en/stable/source/xrpl.utils.html",
                # context for answering questions related to the XRPL
                "https://xrpl.org/docs/introduction/what-is-the-xrp-ledger",
                "https://xrpl.org/docs/introduction/what-is-xrp",
                "https://xrpl.org/docs/introduction/crypto-wallets",
                "https://xrpl.org/docs/introduction/transactions-and-requests",
                # context for answering questions related to the THENA protocol
               "https://docs.thena.fi/thena?ref=bnbchain.ghost.io",
               "https://docs.thena.fi/thena/the-onboarding",
               "https://docs.thena.fi/thena/the-spot-dex/swap-guide",
               "https://docs.thena.fi/thena/the-spot-dex/limit-order",
               "https://docs.thena.fi/thena/the-liquidity-pools/introduction-to-fusion",
               "https://docs.thena.fi/thena/the-liquidity-pools/liquidity-pools-typology",
               "https://docs.thena.fi/thena/the-liquidity-pools/earn-the",
               "https://docs.thena.fi/thena/the-liquidity-pools/earn-trading-fees",
                ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer( 
            # Only keep post title, headers, and content from the full HTML.
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

### Retrieval and generation chain for taking user query at run time and retrieving data from index

1. Use a retriever to match against user query. Extending this to a tool allows models to rewrite user queries into more effective search queries. This also gives model a choice to either respond immediately or do RAG.
2. ChatModel / LLM produces answer from prompt (which takes in user query and retrieved data)

In [ ]:

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(
        query, 
        k=2
    )
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])

## Agent
Now the control flow is defined by the reasoning capabilities of LLMs. Using agents allows you to offload additional discretion over the retrieval process. Although their behavior is less predictable than the above "chain", they are able to execute multiple retrieval steps in service of a query, or iterate on a single search.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver() # support multiple conversational turns

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

websearch = TavilySearchResults(max_results=2)

mem_agent = create_react_agent(llm, [retrieve, websearch], checkpointer=memory)
# display(Image(agent_executor.get_graph().draw_mermaid_png()))

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate, SystemMessagePromptTemplate

def create_tool_agent(llm: ChatVertexAI, tools: list, system_prompt: str):
    """Helper function to create agents with custom tools and system prompt
    Args:
        llm (ChatVertexAI): LLM for the agent
        tools (list): list of tools the agent will use
        system_prompt (str): text describing specific agent purpose

    Returns:
        executor (AgentExecutor): Runnable for the agent created.
    """
    
    # Each worker node will be given a name and some tools.
    
    system_prompt_template = PromptTemplate(

                template= system_prompt + """
                ONLY respond to the part of query relevant to your purpose.
                IGNORE tasks you can't complete. 
                Use the following context to answer your query 
                if available: \n {agent_history} \n
                """,
                input_variables=["agent_history"],
            )

    #define system message
    system_message_prompt = SystemMessagePromptTemplate(prompt=system_prompt_template)

    prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt,
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, 
                return_intermediate_steps= True, verbose = False)
    return executor

## XRPL DEX Setup for agent tooling

In [ ]:
from xrpl.clients import JsonRpcClient
from xrpl.clients import WebsocketClient
from xrpl.wallet import generate_faucet_wallet
from xrpl.models.transactions import Payment
from xrpl.utils import xrp_to_drops
from xrpl.transaction import submit_and_wait
from xrpl.models.requests.account_info import AccountInfo
from xrpl.models.transactions import OfferCreate, OfferCancel

In [ ]:
def get_json_rpc_client():
    """Create a JSON-RPC client for the XRP Ledger."""
    return JsonRpcClient("https://s.altnet.rippletest.net:51234/")

def get_websocket_client():
    """Create a WebSocket client for the XRP Ledger."""
    return WebsocketClient("wss://s.altnet.rippletest.net:51233")

def generate_wallet(client):
    """Generate a new wallet using the XRP Ledger."""
    return generate_faucet_wallet(client, debug=False)

def sign_and_submit_payment_txn(client, wallet, destination, amount):
    """Sign and submit a payment transaction to the XRP Ledger."""
    payment = Payment(
        account=wallet.classic_address,
        amount=xrp_to_drops(amount),
        destination=destination
    )
    response = submit_and_wait(payment, client, wallet)
    return response

def get_account_info(client, address):
    """Get account information from the XRP Ledger."""
    info = AccountInfo(
        account=address,
        edger_index="validated",
        strict=True,
    )
    response = client.request(info)
    return response
    
### Trading specific functions
# https://xrpl.org/docs/tutorials/how-tos/use-tokens/trade-in-the-decentralized-exchange#interactive-lookupoffers
# https://xrpl-py.readthedocs.io/en/stable/source/xrpl.models.transactions.html -> look for OfferCreate and OfferCancel


## Tutorial code

In [ ]:
import asyncio
import pprint
from decimal import Decimal

from xrpl.asyncio.clients import AsyncWebsocketClient
from xrpl.asyncio.transaction import (
    autofill_and_sign,
    submit_and_wait,
)
from xrpl.asyncio.wallet import generate_faucet_wallet
from xrpl.models.currencies import (
    IssuedCurrency,
    XRP,
)
from xrpl.models.requests import (
    AccountLines,
    AccountOffers,
    BookOffers,
)
from xrpl.models.transactions import OfferCreate
from xrpl.utils import (
    drops_to_xrp,
    get_balance_changes,
    xrp_to_drops,
)

In [ ]:
# Get credentials from the Testnet Faucet -----------------------------------
# note: https://xrpl.org/docs/concepts/transactions/secure-signing is the recommended way to sign.
# Testnet info can possibly be reset at any time.
# Address: rGNh4AB95PZ1XrE8BSGDpGWE4n5cFsX1ha
# Secret: sEd7mPHej64humD6kPwvxJYGpFX8axn

async def main():
    async with AsyncWebsocketClient("wss://s.altnet.rippletest.net:51233") as client:

      print("Requesting addresses from the Testnet faucet...")
      wallet = await generate_faucet_wallet(client, debug=True)


    # after exiting the context, the client is closed

asyncio.run(main())



In [ ]:
# Define the proposed trade. ------------------------------------------------
      # Technically you don't need to specify the amounts (in the "value" field)
      # to look up order books using book_offers, but for this tutorial we reuse
      # these variables to construct the actual Offer later.
      #
      # Note that XRP is represented as drops, whereas any other currency is
      # represented as a decimal value.
      we_want = {
          "currency": IssuedCurrency(
              currency="TST",
              issuer="rP9jPyP5kyvFRb6ZiRghAGw5u8SGAmU4bd"
          ),
          "value": "25",
      }

      we_spend = {
          "currency": XRP(),
          # 25 TST * 10 XRP per TST * 15% financial exchange (FX) cost
          "value": xrp_to_drops(25 * 10 * 1.15),
      }

      # "Quality" is defined as TakerPays / TakerGets. The lower the "quality"
      # number, the better the proposed exchange rate is for the taker.
      # The quality is rounded to a number of significant digits based on the
      # issuer's TickSize value (or the lesser of the two for token-token trades).
      proposed_quality = Decimal(we_spend["value"]) / Decimal(we_want["value"])

      # Look up Offers. -----------------------------------------------------------
      # To buy TST, look up Offers where "TakerGets" is TST:
      print("Requesting orderbook information...")
      orderbook_info = await client.request(
          BookOffers(
              taker=wallet.address,
              ledger_index="current",
              taker_gets=we_want["currency"],
              taker_pays=we_spend["currency"],
              limit=10,
          )
      )
      print(f"Orderbook:\n{pprint.pformat(orderbook_info.result)}")

      # Estimate whether a proposed Offer would execute immediately, and...
      # If so, how much of it? (Partial execution is possible)
      # If not, how much liquidity is above it? (How deep in the order book would
      # other Offers have to go before ours would get taken?)
      # Note: These estimates can be thrown off by rounding if the token issuer
      # uses a TickSize setting other than the default (15). In that case, you
      # can increase the TakerGets amount of your final Offer to compensate.

      offers = orderbook_info.result.get("offers", [])
      want_amt = Decimal(we_want["value"])
      running_total = Decimal(0)
      if len(offers) == 0:
          print("No Offers in the matching book. Offer probably won't execute immediately.")
      else:
          for o in offers:
              if Decimal(o["quality"]) <= proposed_quality:
                  print(f"Matching Offer found, funded with {o.get('owner_funds')} "
                        f"{we_want['currency']}")
                  running_total += Decimal(o.get("owner_funds", Decimal(0)))
                  if running_total >= want_amt:
                      print("Full Offer will probably fill")
                      break
              else:
                  # Offers are in ascending quality order, so no others after this
                  # will match either
                  print("Remaining orders too expensive.")
                  break

          print(f"Total matched: {min(running_total, want_amt)} {we_want['currency']}")
          if 0 < running_total < want_amt:
              print(f"Remaining {want_amt - running_total} {we_want['currency']} "
                    "would probably be placed on top of the order book.")

      if running_total == 0:
          # If part of the Offer was expected to cross, then the rest would be placed
          # at the top of the order book. If none did, then there might be other
          # Offers going the same direction as ours already on the books with an
          # equal or better rate. This code counts how much liquidity is likely to be
          # above ours.
          #
          # Unlike above, this time we check for Offers going the same direction as
          # ours, so TakerGets and TakerPays are reversed from the previous
          # book_offers request.

          print("Requesting second orderbook information...")
          orderbook2_info = await client.request(
              BookOffers(
                  taker=wallet.address,
                  ledger_index="current",
                  taker_gets=we_spend["currency"],
                  taker_pays=we_want["currency"],
                  limit=10,
              )
          )
          print(f"Orderbook2:\n{pprint.pformat(orderbook2_info.result)}")

          # Since TakerGets/TakerPays are reversed, the quality is the inverse.
          # You could also calculate this as 1 / proposed_quality.
          offered_quality = Decimal(we_want["value"]) / Decimal(we_spend["value"])

          tally_currency = we_spend["currency"]
          if isinstance(tally_currency, XRP):
              tally_currency = f"drops of {tally_currency}"

          offers2 = orderbook2_info.result.get("offers", [])
          running_total2 = Decimal(0)
          if len(offers2) == 0:
              print("No similar Offers in the book. Ours would be the first.")
          else:
              for o in offers2:
                  if Decimal(o["quality"]) <= offered_quality:
                      print(f"Existing offer found, funded with {o.get('owner_funds')} "
                            f"{tally_currency}")
                      running_total2 += Decimal(o.get("owner_funds", Decimal(0)))
                  else:
                      print("Remaining orders are below where ours would be placed.")
                      break

              print(f"Our Offer would be placed below at least {running_total2} "
                    f"{tally_currency}")
              if 0 < running_total2 < want_amt:
                  print(f"Remaining {want_amt - running_total2} {tally_currency} "
                        "will probably be placed on top of the order book.")

In [ ]:
# Send OfferCreate transaction ----------------------------------------------

        # For this tutorial, we already know that TST is pegged to
        # XRP at a rate of approximately 10:1 plus spread, so we use
        # hard-coded TakerGets and TakerPays amounts.

        tx = OfferCreate(
            account=wallet.address,
            taker_gets=we_spend["value"],
            taker_pays=we_want["currency"].to_amount(we_want["value"]),
        )

        # Sign and autofill the transaction (ready to submit)
        signed_tx = await autofill_and_sign(tx, client, wallet)
        print("Transaction:", signed_tx)

        # Submit the transaction and wait for response (validated or rejected)
        print("Sending OfferCreate transaction...")
        result = await submit_and_wait(signed_tx, client)
        if result.is_successful():
            print(f"Transaction succeeded: "
                  f"https://testnet.xrpl.org/transactions/{signed_tx.get_hash()}")
        else:
            raise Exception(f"Error sending transaction: {result}")

In [ ]:
        # wait for validation 4-7 seconds

# Check metadata of validated transaction's metadata, instead of tentative txn metadata which can be different from final result ------------------------------------------------------------

# In case of an OfferCreate transaction, likely results include:

# Some or all of the Offer may have been filled by matching with existing Offers in the ledger.
# The unmatched remainder, if any, has been placed into the ledger to await new matching Offers.
# Other bookkeeping may have occurred, such as removing expired or unfunded Offers that would have matched.
        
        balance_changes = get_balance_changes(result.result["meta"])
        print(f"Balance Changes:\n{pprint.pformat(balance_changes)}")

        # For educational purposes the transaction metadata is analyzed manually in the
        # following section. However, there is also a get_order_book_changes(metadata)
        # utility function available in the xrpl library, which is generally the easier
        # and preferred choice for parsing the metadata and computing orderbook changes.

        # Helper to convert an XRPL amount to a string for display
        def amt_str(amt) -> str:
            if isinstance(amt, str):
                return f"{drops_to_xrp(amt)} XRP"
            else:
                return f"{amt['value']} {amt['currency']}.{amt['issuer']}"

        offers_affected = 0
        for affnode in result.result["meta"]["AffectedNodes"]:
            if "ModifiedNode" in affnode:
                if affnode["ModifiedNode"]["LedgerEntryType"] == "Offer":
                    # Usually a ModifiedNode of type Offer indicates a previous Offer that
                    # was partially consumed by this one.
                    offers_affected += 1
            elif "DeletedNode" in affnode:
                if affnode["DeletedNode"]["LedgerEntryType"] == "Offer":
                    # The removed Offer may have been fully consumed, or it may have been
                    # found to be expired or unfunded.
                    offers_affected += 1
            elif "CreatedNode" in affnode:
                if affnode["CreatedNode"]["LedgerEntryType"] == "RippleState":
                    print("Created a trust line.")
                elif affnode["CreatedNode"]["LedgerEntryType"] == "Offer":
                    offer = affnode["CreatedNode"]["NewFields"]
                    print(f"Created an Offer owned by {offer['Account']} with "
                          f"TakerGets={amt_str(offer['TakerGets'])} and "
                          f"TakerPays={amt_str(offer['TakerPays'])}.")

        print(f"Modified or removed {offers_affected} matching Offer(s)")

        

In [ ]:
        # Check balances ------------------------------------------------------------
        print("Getting address balances as of validated ledger...")
        balances = await client.request(
            AccountLines(
                account=wallet.address,
                ledger_index="validated",
            )
        )
        pprint.pp(balances.result)

        # Check Offers --------------------------------------------------------------
        print(f"Getting outstanding Offers from {wallet.address} "
              f"as of validated ledger...")
        acct_offers = await client.request(
            AccountOffers(
                account=wallet.address,
                ledger_index="validated",
            )
        )
        pprint.pp(acct_offers.result)

    

## Continuation of agent prompts

In [ ]:
sentiment_prompt = """ You are a assistant for performing research on market sentiment of a cryto asset.
        You are to gauge whether the sentiment is positive, negative or neutral based on news and statements
        made by influential sources in the cryptocurrency space. Also, weigh the sentiment based on the profile of the source.
        For example, a statement from a well-known cryptocurrency influencer may have more weight than a random twitter user with less than average number of followers.
        You must also take into account of whether the opinion is based on facts or speculation, and if the source has a history of being accurate.
        Use your tools to answer questions. If you do not have a tool to
        answer the question, say so. """

sentiment_agent = create_tool_agent(llm=llm, tools = [retrieve, websearch], # todo: add twitter, governance, news tools
              system_prompt = sentiment_prompt)

risk_prompt = """ You are a assistant for performing risk analysis on a crypto asset.
        You are to assess the risk of investing in a cryptocurrency based on the information available.
        You must consider the technology behind the cryptocurrency, the team behind the project, the market conditions, and the regulatory environment.
        Use your tools to complete requests. If you do not have a tool to
       complete the request, say so. """

risk_agent = create_tool_agent(llm=llm, tools = [retrieve, websearch], 
                    system_prompt = risk_prompt)


thena_api_prompt = """ You are an assistant for performing action such as querying user wallet status, trading, swapping and liquidity provision to THENA blockchain ecosystem based on the user's request if any.
        Use your tools to complete requests. If you do not have a tool to
       complete the request, say so. """

thena_api_agent = create_tool_agent(llm=llm, tools = [], 
                    system_prompt = thena_api_prompt)


xrpl_api_prompt = """ You are an assistant for performing action such as querying user wallet status, trading, swapping and liquidity provision to Ripple Ledger (XRPL) blockchain ecosystem based on the user's request if any.
        Use your tools to complete requests. If you do not have a tool to
       complete the request, say so. """

xrpl_api_agent = create_tool_agent(llm=llm, tools = [], # tools for staking, LPing, swapping etc
                    system_prompt = xrpl_api_prompt)

In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

system_prompt_template = PromptTemplate(

      template= """ You are a helpful assistant that summarises agent history 
                      in response to the original user query below. 
                      SUMMARISE ALL THE OUTPUTS AND TOOLS USED in agent_history.
                      The agent history is as follows: 
                        \n{agent_history}\n""",
                input_variables=["agent_history"],  )

system_message_prompt = SystemMessagePromptTemplate(prompt=system_prompt_template)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message_prompt,
        MessagesPlaceholder(variable_name="messages"),
    ])

comms_agent = (prompt| llm) 

In [ ]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Can you do an interactive analysis for what cryptocurrency I should buy?"
                )
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser
from enum import Enum
members = ["Sentiment", "Risk", "THENA_API", "XRPL_API", "Mem", "Communicate"]

#create options map for the supervisor output parser.
member_options = {member:member for member in members}

#create Enum object
MemberEnum = Enum('MemberEnum', member_options)

from pydantic import BaseModel

#force Supervisor to pick from options defined above
# return a dictionary specifying the next agent to call 
#under key next.
class SupervisorOutput(BaseModel):
    #defaults to communication agent
    next: MemberEnum = MemberEnum.Communicate


system_prompt = (
    """You are a supervisor tasked with managing a conversation between the
    crew of workers:  {members}. Given the following user request, 
    and crew responses respond with the worker to act next.
    Each worker will perform a task and respond with their results and status. 
    When finished with the task, route to communicate to deliver the result to 
    user. Given the conversation and crew history below, who should act next?
    Hint: API agents should take into account of sentiment and risk analysis.
    Analysis should take into account mem agent history. 
    Treat mem agent like representative of user and their portfolio.
    Select one of: {options} 
    \n{format_instructions}\n"""
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed

# Using openai function calling can make output parsing easier for us
supervisor_parser = JsonOutputParser(pydantic_object=SupervisorOutput)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_history")
       
    ]
).partial(options=str(members), members=", ".join(members), 
    format_instructions = supervisor_parser.get_format_instructions())


supervisor_chain = (
    prompt | llm |supervisor_parser
)

## Graph

In [ ]:

from langchain_core.messages import AIMessage
import operator

# For agents in the crew 
def crew_nodes(state, crew_member, name):
    #read the last message in the message history.
    input = {'messages': [state['messages'][-1]], 
                'agent_history' : state['agent_history']}
    result = crew_member.invoke(input)
    #add response to the agent history.
    return {"agent_history": [AIMessage(content= result["output"], 
              additional_kwargs= {'intermediate_steps' : result['intermediate_steps']}, 
              name=name)]}

def comms_node(state):
    #read the last message in the message history.
    input = {'messages': [state['messages'][-1]],
                     'agent_history' : state['agent_history']}
    result = comms_agent.invoke(input)
    #respond back to the user.
    return {"messages": [result]}

# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str 
    agent_history: Annotated[Sequence[BaseMessage], operator.add]

In [ ]:
from functools import partial

workflow = StateGraph(AgentState)

mem_node = partial(crew_nodes, crew_member=mem_agent, name="Memory")
sentiment_node = partial(crew_nodes, crew_member=sentiment_agent, name="Sentiment")
risk_node = partial(crew_nodes, crew_member=risk_agent, name="Risk")
thena_api_node = partial(crew_nodes, crew_member=thena_api_agent, name="THENA_API")
xrpl_api_node = partial(crew_nodes, crew_member=xrpl_api_agent, name="XRPL_API")

workflow.add_node("Mem", mem_node)
workflow.add_node("Sentiment", sentiment_node)
workflow.add_node("Risk", risk_node)
workflow.add_node("THENA_API", thena_api_node)
workflow.add_node("XRPL_API", xrpl_api_node)
workflow.add_node("Communicate", comms_node )
workflow.add_node("Supervisor", supervisor_chain)
workflow.set_entry_point("Supervisor")
workflow.add_edge('Mem', "Supervisor")
workflow.add_edge('Sentiment', "Supervisor")
workflow.add_edge('Risk', "Supervisor")
workflow.add_edge('THENA_API', "Supervisor")
workflow.add_edge('XRPL_API', "Supervisor")
workflow.add_edge('Communicate', END) 
# end loop at communication agent.

# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
workflow.add_conditional_edges("Supervisor", lambda x: x["next"], member_options)

graph = workflow.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

## Tests / Examples

In [ ]:
from langchain_core.messages import HumanMessage

for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Can you perform an interactive analysis for what cryptocurrency I should buy?")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")